## Segmenting and Clustering Neighborhoods in Toronto Assignment

#### Question 1 - Website scraping

In [1]:
import pandas as pd

In [2]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
print(len(data))

3


In [4]:
#check which series holds the table of postal codes
data[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
#convert into Pandas dataframe
postal_codes = pd.DataFrame(data[0])

In [6]:
#Remove an "Not assigned" boroughs
postal_codes = postal_codes.drop(postal_codes[postal_codes["Borough"] == 'Not assigned'].index)

#reset the dataframe index
postal_codes.reset_index(drop=True, inplace=True)
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
postal_codes.shape

(103, 3)

### Question 2

In [8]:
import numpy as np

In [9]:
#download csv file
import wget
wget.download('http://cocl.us/Geospatial_data', 'long_lat_data_Toronto.csv')

100% [................................................................................] 2891 / 2891

'long_lat_data_Toronto (1).csv'

In [10]:
#import data from csv into dataframe
long_lats = pd.read_csv('long_lat_data_Toronto.csv')
long_lats.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#merge csv data with original postal code data
postal_codes = postal_codes.join(long_lats.set_index('Postal Code'), on='Postal Code')
postal_codes.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Question 3

In [12]:
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [13]:
# pulling venue data from Foursquare

In [14]:
CLIENT_ID = 'LKRBWXNWBXJRACJHS4CGS03A4SMSRTFUJRK5KYC2U4151CUW' # your Foursquare ID
CLIENT_SECRET = 'IJXTWYSMLHBBKD1E5NADM2R54COFTFLH3ES2MULIADDTWCNC' # your Foursquare Secret
VERSION = '20180605'
radius=500
LIMIT=100

In [15]:
#create function to pull from Foursquare API to get top 100 venue information within 500 meters of the latitude and longitude passed in
def getNearbyVenues(pcodes, names, latitudes, longitudes, radius=500):  
    venues_list=[]
    for pcode, name, lat, lng in zip(pcodes, names, latitudes, longitudes):   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            pcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Borough', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# run function on all Toronto Postal Codes
toronto_venues = getNearbyVenues(pcodes=postal_codes['Postal Code'],
                                   names=postal_codes['Borough'],
                                   latitudes=postal_codes['Latitude'],
                                   longitudes=postal_codes['Longitude']
                                  )
toronto_venues.head()

,Postal Code,Borough,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,North York,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [17]:
# use one hot encoding to transform the venue categorys (which are strings) into a list of columns with binary values
# one hot encoding the toronto venues dataframe
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Postal Code
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M3A
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M3A
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4A
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4A
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4A


In [18]:
# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [19]:
#group by postal code and average values
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#function to populate a dataframe with the most popular venues for each postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
#sort dataframe to get top 5 venue categories per postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pcode_venues_sorted = pd.DataFrame(columns=columns)
pcode_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

In [22]:
for ind in np.arange(toronto_grouped.shape[0]):
    pcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pcode_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,Mexican Restaurant,Electronics Store,Restaurant,Breakfast Spot,Rental Car Location,Medical Center,Bank,Intersection,Yoga Studio,Discount Store
3,M1G,Coffee Shop,Soccer Field,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M1H,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Discount Store,Dim Sum Restaurant


#### K clustering

In [23]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 9, 1, 4, 9, 1, 1, 1])

In [24]:
# add clustering labels
pcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pcode_venues_sorted['Cluster Labels'] = pcode_venues_sorted['Cluster Labels'].astype(int)
pcode_venues_sorted.head()

,Cluster Labels,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,0,M1C,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,1,M1E,Mexican Restaurant,Electronics Store,Restaurant,Breakfast Spot,Rental Car Location,Medical Center,Bank,Intersection,Yoga Studio,Discount Store
3,9,M1G,Coffee Shop,Soccer Field,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,1,M1H,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Discount Store,Dim Sum Restaurant


In [25]:
toronto_merged = postal_codes

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(pcode_venues_sorted.set_index('Postal Code'), on='Postal Code')

In [26]:
#Drop any NaN values and convert to int
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged# check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,9,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Furniture / Home Store,Women's Store,Vietnamese Restaurant,Convenience Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,8,River,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Department Store
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,9,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Bubble Tea Shop,Dance Studio,Mediterranean Restaurant,Men's Store
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market


In [27]:
# create map
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In case the map doesn't render, this is a screenshot of it:

![Image](img/Week3-Question3.PNG)

We can see that there is a large cluster (cluster 9) centered around downtown, implying that the venues in downtown are more similar than the ones in the suburbs.

Now we have each of the clusters and their top 10 most common venues.

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1C,Scarborough,0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,1,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
8,M4B,East York,1,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Breakfast Spot,Bank,Gym / Fitness Center,Antique Shop
14,M4C,East York,1,Pharmacy,Park,Skating Rink,Beer Store,Dance Studio,Video Store,Athletics & Sports,Curling Ice,Bus Stop,Distribution Center
18,M1E,Scarborough,1,Mexican Restaurant,Electronics Store,Restaurant,Breakfast Spot,Rental Car Location,Medical Center,Bank,Intersection,Yoga Studio,Discount Store
25,M6G,Downtown Toronto,1,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store
26,M1H,Scarborough,1,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Discount Store,Dim Sum Restaurant
27,M2H,North York,1,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
29,M4H,East York,1,Sandwich Place,Indian Restaurant,Gas Station,Bank,Intersection,Restaurant,Discount Store,Fast Food Restaurant,Burger Joint,Pizza Place
31,M6H,West Toronto,1,Bakery,Pizza Place,Pharmacy,Middle Eastern Restaurant,Bar,Supermarket,Café,Bank,Grocery Store,Brewery
44,M1L,Scarborough,1,Bakery,Bus Line,Soccer Field,Park,Metro Station,Bus Station,Intersection,Ice Cream Shop,Coworking Space,Convention Center


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,M4J,East York,2,Park,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
52,M2M,North York,2,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
64,M9N,York,2,Park,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,M9M,North York,3,Fabric Shop,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
101,M8Y,Etobicoke,3,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,4,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
10,M6B,North York,4,Park,Metro Station,Pizza Place,Japanese Restaurant,Pub,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
32,M1J,Scarborough,4,Pizza Place,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
50,M9L,North York,4,Pizza Place,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
70,M9P,Etobicoke,4,Pizza Place,Middle Eastern Restaurant,Intersection,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner
77,M9R,Etobicoke,4,Pizza Place,Mobile Phone Shop,Bus Line,Sandwich Place,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
93,M8W,Etobicoke,4,Pizza Place,Pharmacy,Sandwich Place,Dance Studio,Coffee Shop,Pub,Gym,Airport Terminal,Falafel Restaurant,Event Space


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,5,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
16,M6C,York,5,Trail,Park,Field,Hockey Arena,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,M6E,York,5,Park,Women's Store,Pool,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
40,M3K,North York,5,Airport,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
66,M2P,North York,5,Park,Construction & Landscaping,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
68,M5P,Central Toronto,5,Jewelry Store,Park,Sushi Restaurant,Trail,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
85,M1V,Scarborough,5,Park,Playground,Coffee Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
91,M4W,Downtown Toronto,5,Park,Trail,Playground,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,M3M,North York,6,Home Service,Baseball Field,Food Truck,Yoga Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
62,M5N,Central Toronto,6,Garden,Music Venue,Home Service,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M9B,Etobicoke,7,Jewelry Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,M8X,Etobicoke,8,River,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Department Store


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,9,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
3,M6A,North York,9,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Furniture / Home Store,Women's Store,Vietnamese Restaurant,Convenience Store
4,M7A,Downtown Toronto,9,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
7,M3B,North York,9,Gym,Café,Athletics & Sports,Caribbean Restaurant,Japanese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
9,M5B,Downtown Toronto,9,Coffee Shop,Clothing Store,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Café,Japanese Restaurant,Diner,Pizza Place,Fast Food Restaurant
13,M3C,North York,9,Beer Store,Coffee Shop,Restaurant,Gym,Asian Restaurant,Supermarket,Discount Store,Dim Sum Restaurant,Italian Restaurant,Japanese Restaurant
15,M5C,Downtown Toronto,9,Café,Coffee Shop,Clothing Store,Restaurant,American Restaurant,Cocktail Bar,Cosmetics Shop,Department Store,Creperie,Beer Bar
17,M9C,Etobicoke,9,Pharmacy,Coffee Shop,Liquor Store,Shopping Plaza,Beer Store,Café,Pizza Place,Pet Store,Dumpling Restaurant,Eastern European Restaurant
19,M4E,East Toronto,9,Coffee Shop,Health Food Store,Asian Restaurant,Pub,Neighborhood,Trail,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant
20,M5E,Downtown Toronto,9,Coffee Shop,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Café,Cocktail Bar,Vegetarian / Vegan Restaurant
